#########################
# İş Problemi
#########################

- Türkiye’nin en büyük online hizmet platformu olan Armut, hizmet verenler ile hizmet almak isteyenleri buluşturmaktadır.
- Bilgisayarın veya akıllı telefonunun üzerinden birkaç dokunuşla temizlik, tadilat, nakliyat gibi hizmetlere kolayca
- ulaşılmasını sağlamaktadır.
- Hizmet alan kullanıcıları ve bu kullanıcıların almış oldukları servis ve kategorileri içeren veri setini kullanarak
- Association Rule Learning ile ürün tavsiye sistemi oluşturulmak istenmektedir.


#########################
# Veri Seti
#########################
# Veri seti müşterilerin aldıkları servislerden ve bu servislerin kategorilerinden oluşmaktadır.
# Alınan her hizmetin tarih ve saat bilgisini içermektedir.

- UserId: Müşteri numarası
- ServiceId: Her kategoriye ait anonimleştirilmiş servislerdir. (Örnek : Temizlik kategorisi altında koltuk yıkama servisi)
- Bir ServiceId farklı kategoriler altında bulanabilir ve farklı kategoriler altında farklı servisleri ifade eder.
- (Örnek: CategoryId’si 7 ServiceId’si 4 olan hizmet petek temizliği iken CategoryId’si 2 ServiceId’si 4 olan hizmet mobilya montaj)
- CategoryId: Anonimleştirilmiş kategorilerdir. (Örnek : Temizlik, nakliyat, tadilat kategorisi)
- CreateDate: Hizmetin satın alındığı tarih




#########################
# GÖREV 1: Veriyi Hazırlama
#########################

- Adım 1: armut_data.csv dosyasınız okutunuz.


- Adım 2: ServisID her bir CategoryID özelinde farklı bir hizmeti temsil etmektedir.
- ServiceID ve CategoryID'yi "_" ile birleştirerek hizmetleri temsil edecek yeni bir değişken oluşturunuz.



- Adım 3: Veri seti hizmetlerin alındığı tarih ve saatten oluşmaktadır, herhangi bir sepet tanımı (fatura vb. ) bulunmamaktadır.
- Association Rule Learning uygulayabilmek için bir sepet (fatura vb.) tanımı oluşturulması gerekmektedir.
- Burada sepet tanımı her bir müşterinin aylık aldığı hizmetlerdir. Örneğin; 7256 id'li müşteri 2017'in 8.ayında aldığı 9_4, 46_4 hizmetleri bir sepeti;
- 2017’in 10.ayında aldığı  9_4, 38_4  hizmetleri başka bir sepeti ifade etmektedir. Sepetleri unique bir ID ile tanımlanması gerekmektedir.
- Bunun için öncelikle sadece yıl ve ay içeren yeni bir date değişkeni oluşturunuz. UserID ve yeni oluşturduğunuz date değişkenini "_"
- ile birleştirirek ID adında yeni bir değişkene atayınız.



#########################
# GÖREV 2: Birliktelik Kuralları Üretiniz
#########################

- Adım 1: Aşağıdaki gibi sepet hizmet pivot table’i oluşturunuz.

- Hizmet         0_8  10_9  11_11  12_7  13_11  14_7  15_1  16_8  17_5  18_4..
- SepetID
- 0_2017-08        0     0      0     0      0     0     0     0     0     0..
- 0_2017-09        0     0      0     0      0     0     0     0     0     0..
- 0_2018-01        0     0      0     0      0     0     0     0     0     0..
- 0_2018-04        0     0      0     0      0     1     0     0     0     0..
- 10000_2017-08    0     0      0     0      0     0     0     0     0     0..




- Adım 2: Birliktelik kurallarını oluşturunuz.



- Adım 3: arl_recommender fonksiyonunu kullanarak en son 2_0 hizmetini alan bir kullanıcıya hizmet önerisinde bulununuz.

In [26]:
# Adım 1: armut_data.csv dosyasınız okutunuz.
import pandas as pd
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
pd.set_option('display.width', 600)
# çıktının tek bir satırda olmasını sağlar.
pd.set_option('display.expand_frame_repr', False)


In [27]:
df_ = pd.read_csv("armut_data.csv")
df = df_.copy()
df.head()

,UserId,ServiceId,CategoryId,CreateDate
0,25446,4,5,2017-08-06 16:11:00
1,22948,48,5,2017-08-06 16:12:00
2,10618,0,8,2017-08-06 16:13:00
3,7256,9,4,2017-08-06 16:14:00
4,25446,48,5,2017-08-06 16:16:00


In [28]:
df.dtypes

UserId         int64
ServiceId      int64
CategoryId     int64
CreateDate    object
dtype: object

In [29]:
# Adım 2: ServisID her bir CategoryID özelinde farklı bir hizmeti temsil etmektedir.
# ServiceID ve CategoryID'yi "_" ile birleştirerek hizmetleri temsil edecek yeni bir değişken oluşturunuz.

df["services"] = df["ServiceId"].astype(str).str.cat(df["CategoryId"].astype(str), sep="_")

df.head()



,UserId,ServiceId,CategoryId,CreateDate,services
0,25446,4,5,2017-08-06 16:11:00,4_5
1,22948,48,5,2017-08-06 16:12:00,48_5
2,10618,0,8,2017-08-06 16:13:00,0_8
3,7256,9,4,2017-08-06 16:14:00,9_4
4,25446,48,5,2017-08-06 16:16:00,48_5


In [30]:
# Adım 3: Veri seti hizmetlerin alındığı tarih ve saatten oluşmaktadır, herhangi bir sepet tanımı (fatura vb. ) bulunmamaktadır.
# Association Rule Learning uygulayabilmek için bir sepet (fatura vb.) tanımı oluşturulması gerekmektedir.
# Burada sepet tanımı her bir müşterinin aylık aldığı hizmetlerdir. Örneğin; 7256 id'li müşteri 2017'in 8.ayında aldığı 9_4, 46_4 hizmetleri bir sepeti;
# 2017’in 10.ayında aldığı  9_4, 38_4  hizmetleri başka bir sepeti ifade etmektedir. Sepetleri unique bir ID ile tanımlanması gerekmektedir.
# Bunun için öncelikle sadece yıl ve ay içeren yeni bir date değişkeni oluşturunuz. UserID ve yeni oluşturduğunuz date değişkenini "_"
# ile birleştirirek ID adında yeni bir değişkene atayınız.

df["new_date"] = pd.to_datetime(df["CreateDate"]).dt.strftime('%Y-%m')

df["Cart_ID"] = df["UserId"].astype(str).str.cat(df["new_date"], sep="_")

df.head()

,UserId,ServiceId,CategoryId,CreateDate,services,new_date,Cart_ID
0,25446,4,5,2017-08-06 16:11:00,4_5,2017-08,25446_2017-08
1,22948,48,5,2017-08-06 16:12:00,48_5,2017-08,22948_2017-08
2,10618,0,8,2017-08-06 16:13:00,0_8,2017-08,10618_2017-08
3,7256,9,4,2017-08-06 16:14:00,9_4,2017-08,7256_2017-08
4,25446,48,5,2017-08-06 16:16:00,48_5,2017-08,25446_2017-08


In [ ]:
#########################
# GÖREV 2: Birliktelik Kuralları Üretiniz
#########################

# Adım 1: Aşağıdaki gibi sepet hizmet pivot table’i oluşturunuz.
# Hizmet         0_8  10_9  11_11  12_7  13_11  14_7  15_1  16_8  17_5  18_4..
# SepetID
# 0_2017-08        0     0      0     0      0     0     0     0     0     0..
# 0_2017-09        0     0      0     0      0     0     0     0     0     0..
# 0_2018-01        0     0      0     0      0     0     0     0     0     0..
# 0_2018-04        0     0      0     0      0     1     0     0     0     0..
# 10000_2017-08    0     0      0     0      0     0     0     0     0     0..

pivot_df = df.groupby(["Cart_ID", "services"]) .count().unstack().fillna(0).applymap(lambda x: 1 if x > 0 else 0)

common_movies.pivot_table(index=["userId"], columns=["title"], values="rating") 

dfp= df.pivot_table(index=["Cart_ID"], columns=["services"], values="count").unstack().fillna(0).applymap(lambda x: 1 if x > 0 else 0)


pivot_df.head()

# "Cart_ID" ve "services" sütunlarına göre verileri grupladık ve her bir grup için "services" sütunundaki hizmetlerin sayısını count  ile saydırdık.
#  Daha sonra bu verileri "unstack" yöntemi kullanarak, "services" sütununda birbirinden farklı olan hizmetleri sütun isimleri yaparak bir pivot tablosu şeklinde düzenlemiş olduk.
# "fillna(0)" yöntemi kullanarak, pivot tablosundaki herhangi bir NaN değerini 0 ile doldurduk.
# "applymap(lambda x: 1 if x > 0 else 0)" koduyla, pivot tablosundaki her bir hücreyi, hizmet'in (services) o sepete(Cart_ID) atanıp atanmadığını gösteren 0 veya 1 şeklinde ifade eder. 
# Eğer hücredeki değer 0'dan büyükse 1, değilse 0 olarak değiştirir.



services,0_8,10_9,11_11,12_7,13_11,14_7,15_1,16_8,17_5,18_4,19_6,1_4,20_5,21_5,22_0,23_10,24_10,25_0,26_7,27_7,28_4,29_0,2_0,30_2,31_6,32_4,33_4,34_6,35_11,36_1,37_0,38_4,39_10,3_5,40_8,41_3,42_1,43_2,44_0,45_6,46_4,47_7,48_5,49_1,4_5,5_11,6_7,7_3,8_5,9_4
Cart_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0_2017-08,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0
0_2017-09,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
0_2018-01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
0_2018-04,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
10000_2017-08,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [32]:
# Adım 2: Birliktelik kurallarını oluşturunuz.
############################################
# 3. Birliktelik Kurallarının Çıkarılması
############################################
import warnings
warnings.filterwarnings("ignore", message="DataFrames with non-bool")

from mlxtend.frequent_patterns import apriori, association_rules

frequent_itemsets = apriori(pivot_df,
                            min_support=0.01,
                            use_colnames=True)

frequent_itemsets.sort_values("support", ascending=False)

from itertools import combinations

rules = association_rules(frequent_itemsets,
                          metric="support",
                          min_threshold=0.01)

rules.head()




,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(2_0),(13_11),0.130286,0.056627,0.012819,0.098394,1.737574,0.005442,1.046325
1,(13_11),(2_0),0.056627,0.130286,0.012819,0.226382,1.737574,0.005442,1.124216
2,(2_0),(15_1),0.130286,0.120963,0.033951,0.260588,2.154278,0.018191,1.188833
3,(15_1),(2_0),0.120963,0.130286,0.033951,0.280673,2.154278,0.018191,1.209066
4,(15_1),(33_4),0.120963,0.027310,0.011233,0.092861,3.400299,0.007929,1.072262


In [43]:
#Adım 3: arl_recommender fonksiyonunu kullanarak en son "2_0 "hizmetini alan bir kullanıcıya hizmet önerisinde bulununuz.(iki öneride bulundum.)

def arl_recommender(rules_df, last_service, rec_count=1):
    sorted_rules = rules_df.sort_values("lift", ascending=False) #büyükten küçüğe sıraladık(birlikte olma durumlarına bakmak istiyorum.
    recommendation_list = []
    for i, Servicess in enumerate(sorted_rules["antecedents"]):
        for j in list(Servicess):
            if j == last_service:
                recommendation_list.append(list(sorted_rules.iloc[i]["consequents"]))
                                           
    return recommendation_list[0:rec_count]

arl_recommender(rules,"2_0",2)





[['22_0'], ['25_0']]

- "Association Rule Learning (ARL)" algoritması kullanarak, önceden belirlenmiş bir hizmeti takip eden diğer
hizmetleri tavsiye etmek için tasarlanmış bir fonksiyon olan "arl_recommender" adlı bir fonksiyon tanımlar
### Fonksiyon, üç parametre alır:
- **"rules_df":** ARL algoritması ile çıkarılan ve kuralların yer aldığı bir veri çerçevesidir.
- **"last_service":** tavsiye edilecek hizmetin takip eden son hizmettir.
- **"rec_count":** tavsiye edilecek hizmet sayısını belirler.
- **NOT:** Lift, iki olay arasındaki ilişkinin ne kadar güçlü olduğunu, bu beklenen frekans değerine oranla ölçer.